In [67]:
# Load the dataset (10/16/2023, Monday, Quirino Avenue)
north_data = pd.read_csv('./data/north_data.csv')
south_data = pd.read_csv('./data/south_data.csv')

In [92]:
# Track the total number of vehicles
total_vehicles_by_type = {}
total_vehicles_percent = {}
total_vehicles = 0
total_percent = 0

print('Total Number of Vehicles (per type)')
for columns in range(15):
    col = str(columns + 1)
    total_vehicles_by_type[col] = north_data[col].sum() + south_data[col].sum()
    print(f'{col}: {total_vehicles_by_type[col]}')
print()

for v in total_vehicles_by_type.values():
    total_vehicles += v

print('Percentage of Vehicle Types')
for k,v in total_vehicles_by_type.items():
    total_vehicles_percent[k] = (v / total_vehicles)
    total_percent += total_vehicles_percent[k]
    print(f'{k}: {total_vehicles_percent[k]:.2f}')
print()

print(f'Total number of vehicles this day: {total_vehicles}\n')
print(f'Percentage: {total_percent}')

Total Number of Vehicles (per type)
1: 8739
2: 25339
3: 2922
4: 5047
5: 481
6: 70
7: 7
8: 115
9: 269
10: 22
11: 0
12: 0
13: 0
14: 0
15: 1484

Percentage of Vehicle Types
1: 0.20
2: 0.57
3: 0.07
4: 0.11
5: 0.01
6: 0.00
7: 0.00
8: 0.00
9: 0.01
10: 0.00
11: 0.00
12: 0.00
13: 0.00
14: 0.00
15: 0.03

Total number of vehicles this day: 44495

Percentage: 1.0


In [96]:
import pandas as pd

# --- 1. CONFIGURATION: YOU MUST EDIT THIS SECTION ---

# Input Data Files
NORTH_CSV = "./data/north_data.csv"  # Your cleaned Northbound data
SOUTH_CSV = "./data/south_data.csv"  # Your cleaned Southbound data

# Output File
OUTPUT_ROUTES_FILE = "traffic_routes.rou.xml"

# Simulation Parameters
SIMULATION_END_TIME = 86400  # 24 hours in seconds

# --- CRITICAL: UPDATE WITH YOUR REAL EDGE IDs FROM NETEDIT ---
# This is the map you created in Step 1.
# The names are examples; yours will be different!
EDGES = {
    "north_entry": "north_start",
    "north_exit": "north_end",
    "south_entry": "south_start",
    "south_exit": "south_end",
    "east_entry": "east_start",
    "east_exit": "east_end",
    "west_entry": "west_start",
    "west_exit": "west_end",
}

# --- 2. NEW TURNING ASSUMPTIONS ---
# (Fraction of vehicles that continue straight vs. turn)
PROB_STRAIGHT = 0.90
PROB_TURN = 0.10
SIDE_ROAD_FACTOR = 0.25 # Side road traffic as a fraction of main road

# --- 3. SCRIPT LOGIC ---
def generate_routes():
    try:
        north_df = pd.read_csv(NORTH_CSV)
        south_df = pd.read_csv(SOUTH_CSV)
        north_hourly = north_df.iloc[:, -1]
        south_hourly = south_df.iloc[:, -1]
    except FileNotFoundError as e:
        print(f"Error: {e}. Make sure your CSV files are in the correct location.")
        return

    with open(OUTPUT_ROUTES_FILE, "w") as routes:
        routes.write("<routes>\n")

        # --- Define the VALID routes based on your list ---
        # Note: These now include intermediate edges. UPDATE THEM!
        routes.write(f'    <route id="N_to_N" edges="{EDGES["north_entry"]} {EDGES["north_exit"]}"/>\n')
        routes.write(f'    <route id="S_to_S" edges="{EDGES["south_entry"]} {EDGES["south_exit"]}"/>\n')
        routes.write(f'    <route id="E_to_E" edges="{EDGES["east_entry"]} {EDGES["east_exit"]}"/>\n')
        routes.write(f'    <route id="W_to_W" edges="{EDGES["west_entry"]} {EDGES["west_exit"]}"/>\n\n')
        
        routes.write(f'    <route id="W_to_N" edges="{EDGES["west_entry"]} {EDGES["north_exit"]}"/>\n')
        routes.write(f'    <route id="E_to_S" edges="{EDGES["east_entry"]} {EDGES["south_exit"]}"/>\n')
        routes.write(f'    <route id="S_to_W" edges="{EDGES["south_entry"]} {EDGES["west_exit"]}"/>\n')
        # Add a route for North to East if that's the correct turn
        routes.write(f'    <route id="N_to_E" edges="{EDGES["north_entry"]} {EDGES["east_exit"]}"/>\n\n')


        # --- Generate hourly flows with correct turning logic ---
        for hour in range(24):
            begin_time = hour * 3600
            end_time = (hour + 1) * 3600

            n_vehicles = north_hourly.get(hour, 0)
            s_vehicles = south_hourly.get(hour, 0)
            avg_main_road = (n_vehicles + s_vehicles) / 2
            ew_vehicles = int(avg_main_road * SIDE_ROAD_FACTOR)

            # Northbound traffic splits between continuing North and turning East
            if n_vehicles > 0:
                routes.write(f'    <flow id="n_n_{hour}" route="N_to_N" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(n_vehicles * PROB_STRAIGHT)}"/>\n')
                routes.write(f'    <flow id="n_e_{hour}" route="N_to_E" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(n_vehicles * PROB_TURN)}"/>\n')
            
            # Southbound traffic splits between continuing South and turning West
            if s_vehicles > 0:
                routes.write(f'    <flow id="s_s_{hour}" route="S_to_S" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(s_vehicles * PROB_STRAIGHT)}"/>\n')
                routes.write(f'    <flow id="s_w_{hour}" route="S_to_W" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(s_vehicles * PROB_TURN)}"/>\n')

            # Eastbound and Westbound traffic
            if ew_vehicles > 0:
                 routes.write(f'    <flow id="e_e_{hour}" route="E_to_E" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(ew_vehicles * PROB_STRAIGHT)}"/>\n')
                 routes.write(f'    <flow id="e_s_{hour}" route="E_to_S" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(ew_vehicles * PROB_TURN)}"/>\n')
                 routes.write(f'    <flow id="w_w_{hour}" route="W_to_W" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(ew_vehicles * PROB_STRAIGHT)}"/>\n')
                 routes.write(f'    <flow id="w_n_{hour}" route="W_to_N" type="realdistribution" begin="{begin_time}" end="{end_time}" number="{int(ew_vehicles * PROB_TURN)}"/>\n')

        routes.write("</routes>\n")

if __name__ == "__main__":
    generate_routes()
    print(f"Successfully generated routes file with CORRECTED paths: {OUTPUT_ROUTES_FILE}")

Successfully generated routes file with CORRECTED paths: traffic_routes.rou.xml


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def analyze_tripinfo(filepath):
    """Parses a tripinfo file and returns a dictionary of key metrics."""
    try:
        df = pd.read_xml(filepath)
        metrics = {
            'Throughput': len(df),
            'Avg. Wait Time (s)': df['waitingTime'].mean(),
            'Avg. Time Loss (s)': df['timeLoss'].mean(),
            'Avg. Trip Duration (s)': df['duration'].mean()
        }
        return metrics
    except FileNotFoundError:
        print(f"Error: {filepath} not found.")
        return None

# --- Main Analysis ---
baseline_metrics = analyze_tripinfo("tripinfo.xml")
rl_agent_metrics = analyze_tripinfo("rl_trip_info.xml")

if baseline_metrics and rl_agent_metrics:
    # 1. Print a comparison table
    print("--- Performance Comparison ---")
    df_compare = pd.DataFrame({'Baseline': baseline_metrics, 'RL Agent': rl_agent_metrics})
    print(df_compare.round(2))

    # 2. Visualize the comparison with a bar chart
    labels = list(baseline_metrics.keys())
    baseline_values = list(baseline_metrics.values())
    rl_values = list(rl_agent_metrics.values())

    x = np.arange(len(labels))
    width = 0.35

    fig, ax = plt.subplots(figsize=(12, 7))
    rects1 = ax.bar(x - width/2, baseline_values, width, label='Baseline (Fixed-Time)')
    rects2 = ax.bar(x + width/2, rl_values, width, label='RL Agent')

    ax.set_ylabel('Scores')
    ax.set_title('Performance Comparison: Baseline vs. RL Agent')
    ax.set_xticks(x)
    ax.set_xticklabels(labels)
    ax.legend()
    ax.bar_label(rects1, padding=3, fmt='%.1f')
    ax.bar_label(rects2, padding=3, fmt='%.1f')
    fig.tight_layout()
    plt.show()

ImportError: lxml not found, please install or use the etree parser.